# Projeto 01 - EDA (Medical Cost Personal Datasets)

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew
from scipy import stats
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white
from statsmodels.formula.api import ols
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set()

## Business Case

Here we have a dataset that contains information about medical costs from an insurance company, containing different kinds of people. Our desire is to understand the data better and gather insights about the field of medical insurance.

We might be able to answer questions such as: 

* Can you accurately predict insurance costs?
* What variables influence the patients' cost the most?
* Can we predict medical insurance charges based on the other variables available?
* How does the insurance charges relate to all the other features?

We can use this dataset to understand how the variables such as age, sex, bmi, children, smoker and region to check how they influence and are correlated with the variable 'charges'. This is important because we might be able to accurately price rates for potential customers, minimizing risk and optimizing profits based on that persons's data.

## Data Collection & Cleaning

In this case the data collection was quite easy. We didn't have to scrape any website, connect to any API or database to gather the data from there. It was just downloading the dataset from kaggle in csv format.

The dataset was also very 'clean' so there wasn't much to do in that area.

In [ ]:
df = pd.read_csv('insurance.csv')

In [ ]:
df

## EDA (Exploratory Data Analysis)

### Univariate Statistics

Here we will check things such as:

* General information about the variables (data type, total values, unique values, missing data)

* Range and central tendencies measures (min, max, mean, median, mode, quartiles)

* Normality and spread (standard deviation, skewness, kurtosis)

When checking the distribution of variables the ideal scenario is to have a perfectly normal distribution (shaped like a bell curve).

**Quick summary of the numerical variables in the dataset**

In [ ]:
df.describe()

**Checking the column names for this dataset**

In [ ]:
df.columns

**Checking for missing values**

We have 1338 rows (observations) in this dataset. Let's check if there are any missing values in any of the columns.

Let's count the number of values the columns mentioned. If it's lesses than 1338, we know there are missing values.

In [ ]:
print(f'age: {df.age.count()}')
print(f'sex: {df.sex.count()}')
print(f'bmi: {df.bmi.count()}')
print(f'children: {df.children.count()}')
print(f'smoker: {df.smoker.count()}')
print(f'region: {df.region.count()}')
print(f'charges: {df.charges.count()}')

Here is another way to check for missing data:

In [ ]:
print(f'age: {df.age.isnull().sum()}')
print(f'sex: {df.sex.isnull().sum()}')
print(f'bmi: {df.bmi.isnull().sum()}')
print(f'children: {df.children.isnull().sum()}')
print(f'smoker: {df.smoker.isnull().sum()}')
print(f'region: {df.region.isnull().sum()}')
print(f'charges: {df.charges.isnull().sum()}')

**How many unique values are in each column?**

In [ ]:
print(f'age: {df.age.nunique()}')
print(f'sex: {df.sex.nunique()}')
print(f'bmi: {df.bmi.nunique()}')
print(f'children: {df.children.nunique()}')
print(f'smoker: {df.smoker.nunique()}')
print(f'region: {df.region.nunique()}')
print(f'charges: {df.charges.nunique()}')

**What are the data types for each variable?**

In [ ]:
print(f'age: {df.age.dtype}')
print(f'sex: {df.sex.dtype}')
print(f'bmi: {df.bmi.dtype}')
print(f'children: {df.children.dtype}')
print(f'smoker: {df.smoker.dtype}')
print(f'region: {df.region.dtype}')
print(f'charges: {df.charges.dtype}')

**Checking to see which variables are numeric**

In [ ]:
print(f'age: {pd.api.types.is_numeric_dtype(df.age)}')
print(f'sex: {pd.api.types.is_numeric_dtype(df.sex)}')
print(f'bmi: {pd.api.types.is_numeric_dtype(df.bmi)}')
print(f'children: {pd.api.types.is_numeric_dtype(df.children)}')
print(f'smoker: {pd.api.types.is_numeric_dtype(df.smoker)}')
print(f'region: {pd.api.types.is_numeric_dtype(df.region)}')
print(f'charges: {pd.api.types.is_numeric_dtype(df.charges)}')

**Looking at the boundaries, range and middle part of the 'charges' variable**

In [ ]:
print(df.charges.min())
print(df.charges.quantile(.25))
print(df.charges.quantile(.50))
print(df.charges.quantile(.75))
print(df.charges.max())
print(df.charges.mean())
print(df.charges.median())
print(df.charges.mode().values[0])

**Looking at the 'spread' (standard deviation) for the 'charges' variable**

The pandas formula assumes that it's a sample, the numpy formula assumes that it's a population, so we need to input a new parameter.

In [ ]:
df.charges.std()

In [ ]:
np.std(df.charges, ddof = 1)

**Checking for skewness and kurtosis**

Scipy assumes population, pandas assumes sample.

Generally speaking, acceptable values for skewness and kurtosis are between -1 and 1.

In [ ]:
print(skew(df.charges, bias = False))
print(kurtosis(df.charges, bias = False))

In [ ]:
print(df.charges.skew())
print(df.charges.kurt())

This means that it's right skewed (longer right tail) and "higher" and not flattened because of the kurtosis.

### Bivariate Statistics

Here we are interested to see the relationship between the dependent variable with the independent ones. In other words, the relationship between the features (independent) and labels (dependent).

The relationships can be: 

* Numerical/numerical (pearson correlation R) + scatterplot
* Numerical/categorical (one way ANOVA) + bar chart
* Categorical/categorical (Pearson chi-square) + crosstab

Pearson Correlation Coef R have assumptions that need to be met in order to be meaningful:

* Continuous data
* Linear relationship
* Homoskedastic (the error is the same across all values of X)

**P-value and Correlation**

P value is the probability that the results we obtained are not going to be repeated in further analysis. It's the likelihood that what we saw is not truly representative of the data.

* Low p value: good, we can trust the results that we've got
* High p value: bad, means that we can't trust the results that we've got

**Correlation Matrix of the Dataset**

In [ ]:
df.corr()

**We can specify the variables we want to see the correlation of**

In [ ]:
df.charges.corr(df.bmi)

**We can also see the correlation plus the p-value between variables**

In [ ]:
r, p = stats.pearsonr(df.charges, df.age)
print(round(r,4))
print(round(p,29))

**Calculating correlation and p-value for charges to all numeric variables in the dataset**

In [ ]:
corr_df = pd.DataFrame(columns=['r','p'])

for col in df:
    if pd.api.types.is_numeric_dtype(df[col]) and col!= 'charges':
        r, p = stats.pearsonr(df.charges, df[col])
        corr_df.loc[col] = [round(r,3), round(p,3)]
        
corr_df

## Visualizations

### Numerical to Numerical Variables

**Scatterplots**

In [ ]:
plt.scatter(df.age, df.charges)
plt.title('Age x Charges')
plt.xlabel('Age')
plt.ylabel('Charges')
plt.show()

Here we can see a correlation between age and charges. There are 3 separate clusters of individuals and the correlation is positive for all of them.

#### Now let's separate the dataframe into smokers and non smokers and plot them together

In [ ]:
df_smoker = df[df['smoker'] == 'yes']
df_nonsmoker = df[df['smoker'] == 'no']

plt.scatter(df_smoker.age, df_smoker.charges, label = 'Smokers')
plt.scatter(df_nonsmoker.age, df_nonsmoker.charges, label = 'NonSmokers')
plt.title('Age x Charges')
plt.xlabel('Age')
plt.ylabel('Charges')
plt.legend()
plt.show()

It's visible that non smokers cost less to the insurance company.

#### Adding a regression line to the scatterplot

In [ ]:
a, b, r, p, err = stats.linregress(df.age, df.charges)

# y = ax + b
# y = slope(x) + intercept

x = range(18,66)
y = a * x + b

plt.plot(x,y, color = 'green')
plt.scatter(df.age, df.charges)
plt.title('Age x Charges')
plt.xlabel('Age')
plt.ylabel('Charges')
plt.show()

Charges go up as the age goes up, and we can see that there are 03 distinct groups as well. In addition, the regression line is located at the bottom because there are many datapoints concentrated in that area.

#### Calculating values for heteroscedasticity in the data (how it's spread)

In [ ]:
model = ols(formula='charges~age', data= df).fit()

white_test = het_white(model.resid, model.model.exog)
bruschpagan_test = het_breuschpagan(model.resid, model.model.exog)

output_df = pd.DataFrame(columns=['LM stat','LM p','F stat','F stat p'])
output_df.loc['White'] = white_test
output_df.loc['Brusch-Pagan'] = bruschpagan_test

output_df

We can see a pattern of relative small F stats and large p-values. This means the difference between the residuals for all values of X is consistent. We use this test to check if we can rely on the predictions across all values of X.

#### Using seaborn we can plot scatterplots combined with the histogram for each variable.

In [ ]:
sns.set(color_codes = True)
sns.jointplot(x='age',y='charges', data=df)
plt.show()

#### Using the hex and kde kinds we can see where the datapoints are more dense and concentrated

In [ ]:
sns.set_style('white')
sns.jointplot(x='age',y='charges', data=df, kind = 'hex')

In [ ]:
sns.jointplot(x='age', y='charges', data=df, kind = 'kde')

In [ ]:
f, ax = plt.subplots(figsize = (8, 6))
cmap = sns.cubehelix_palette(as_cmap = True, dark = 0, light = 1, reverse = False)
sns.kdeplot(df.age, df.charges, cmap = cmap, n_levels = 60, shade = True)
plt.show()

**Comparing the variables all at once**

In [ ]:
sns.pairplot(df)

In [ ]:
g = sns.PairGrid(df)
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.kdeplot, n_levels=10)

**Visualizing in multiple dimensions**

In [ ]:
font = {'size': 8}
plt.rc('font', **font)

fig = plt.figure()
three_d_plot = Axes3D(fig)
three_d_plot.scatter(df.age, df.bmi, df.charges)

plt.show()

**Making it more interactive!**

In [ ]:
fig = px.scatter_3d(df, x='age', y='charges', z='bmi', color = 'smoker', symbol = 'sex')
fig.show()

### Numerical to Categorical Variables

Assumptions concerning ANOVAs and t-tests (for each group of the categorical variable):
* Normal distribution of the numeric variable
* Equal variance of the numerical variable
* Data used was sampled independently between the groups

In [ ]:
sns.histplot(data = df, x='charges', hue='sex', kde=True);

In [ ]:
sex_m = df[df['sex'] == 'male']
sex_f = df[df['sex'] == 'female']

stats.ttest_ind(sex_m['charges'], sex_f['charges'])

In [ ]:
sns.histplot(data = df, x='charges', hue='smoker', kde=True);

In [ ]:
smoker_y = df[df['smoker'] == 'yes']
smoker_n = df[df['smoker'] == 'no']

stats.ttest_ind(smoker_y['charges'], smoker_n['charges'])

**Automating this task**

In [ ]:
def anova(feature, label):
    groups = df[feature].unique()
    grouped_values = []
    for group in groups:
        grouped_values.append(df[df[feature] == group][label])
    return stats.f_oneway(*grouped_values)

In [ ]:
anova('smoker','charges')

## Multiple Linear Regression

In [ ]:
label = 'charges'

y = df.charges
X = df[['age', 'bmi', 'children']].assign(const=1)

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

**Creating a column in the dataframe with the predictions the model has calculated**

In [ ]:
df['predictions'] = results.fittedvalues
df

R squared has a pretty low value so the predictions are not accurate. Let's try putting in the categorical variables as independent variables as well. But first we need to make a few changes (dummy variables).

In [ ]:
df = pd.get_dummies(df, columns=['sex'], prefix = 'sex', drop_first=True)
df = pd.get_dummies(df, columns=['smoker'], prefix = 'smoker', drop_first=True)
df = pd.get_dummies(df, columns=['region'], prefix = 'region', drop_first=True)

df.head()

In [ ]:
df.head()

In [ ]:
y = df.charges
X = df.drop(columns=[label, 'predictions']).assign(const=1)

results = sm.OLS(y, X).fit()
print(results.summary())

With the categorical variables turned into dummy, in order to use them in the model, we can see that the R squared has significantly gone up.

Interpretation of coefficients: for every 01 year anyone ages, their charges increase by 256 dollars +-.

Now we want to make the coefficients comparable, but they are in different scales... So we need to normalize the scales of these values.

In [ ]:
df_zscore = pd.DataFrame(preprocessing.StandardScaler().fit_transform(df), columns=df.columns)
df_zscore.head()

In [ ]:
y = df_zscore.charges
X = df_zscore.drop(columns=['predictions', 'charges']).assign(const=1)

results = sm.OLS(y, X).fit()
print(results.summary())

In [ ]:
sns.histplot(y);

In [ ]:
sns.histplot(df.charges);

Normalizing does not change the distributions of the variables, neither the predictions we make. But the coefficients become comparable and interpretable.

**MinMax Normalization**

In [ ]:
df_minmax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df), columns=df.columns)
df_minmax.head()

Setting the min values to 0 and the max values to 1, making it even easier to interpret the coefficients.

In [ ]:
sns.histplot(df_minmax.charges);

In [ ]:
y = df_minmax.charges
X = df_minmax.drop(columns=['predictions', 'charges']).assign(const=1)

results = sm.OLS(y, X).fit()
print(results.summary())

The skew is high (>1) so let's make more adjustments to the dataframe. Let's apply a natural log transformation.

In [ ]:
y = np.log1p(y)
sns.histplot(y);

Not quite normally distributed yet. Let's try applying the transformation before the standardization.

In [ ]:
sns.histplot(np.log(df.charges));

Much better (:

In [ ]:
y = np.log(df.charges)
X = df.drop(columns=['predictions','charges']).assign(const=1)

print(sm.OLS(y, X).fit().summary())

**Multicolinearity**

It's when we have independent variables that are highly correlated to each other.

In [ ]:
# VIF = variance inflation factor = 1 / (1 - R2)

def vif(df):
    import pandas as pd
    from sklearn.linear_model import LinearRegression
    
    # dictionaries
    vif_dict, tolerance_dict = {}, {}
    
    # form input data for each indep variable
    for col in df.drop(columns=['const']):
        y = df[col]
        X = df.drop(columns=[col])
        
        # extract r2 from the fit
        r_squared = LinearRegression().fit(X, y).score(X, y)
        
        # calculate vif
        if r_squared < 1:
            vif = 1/(1 - r_squared)
        else:
            vif = 100
        vif_dict[col] = vif
            
        # calculate tolerance
        tolerance = 1 - r_squared
        tolerance_dict[col] = tolerance
        
        # generate the dataframe
        df_output = pd.DataFrame({'VIF': vif_dict, 'Tolerance' : tolerance_dict})
        
    return df_output.sort_values(by=['VIF'], ascending=False)

vif(X)

For VIF values:

* < 10 ADEQUATE
* < 5 GOOD
* < 3 IDEAL